### Define the Prompt

We define a prompt that can perform two tasks given a topic and an argument <br>
1) classify whether the argument is a fallacy, and if so what fallacy the  argument contains <br>
2) decide whether the argument supports or refutes the topic <br>
Prompts templates are defined by a string where ```{}``` is a placeholder for where the inputs for a sample are to be and ```<mask>``` is filled in as the predicted label. <br>
*Multiple masked tokens in a single prompt is currently not supported*

In [ ]:
from prompt import Prompt, RobertaPrompt

In [93]:
templates = {"fallacy": "fallacy task. Topic: {} Text: {} This contains the fallacy: <mask>", 
             "stance": "procon task. Topic: {} Text: {} Has the relation: <mask>"}

def fallacy_policy(pred):
  fallacies = {'AppealtoEmotion', 'RedHerring', 'NoFallacy', 'IrrelevantAuthority','AdHominem','HastyGeneralization'}
  if pred not in fallacies: return 'UNKNOWN'
  return pred
def stance_policy(pred):
  if pred not in {"support", "contradict"}: return "UNKNOWN"
  return pred

policies = {"fallacy": fallacy_policy, "stance": stance_policy}

argument_prompt = Prompt(templates, policies)

RobertaPrompt uses this class to convert a sample into it's desire prompt - for example

In [94]:
argument_prompt.test_sample(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "fallacy")

'fallacy task. Topic: Should we allow animal testing? Text: Animal testing abuses animals and should be dis-continued This contains the fallacy: <mask>'

In [95]:
argument_prompt.train_sample(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "NoFallacy", "fallacy")

'fallacy task. Topic: Should we allow animal testing? Text: Animal testing abuses animals and should be dis-continued This contains the fallacy: NoFallacy'

In [96]:
print(argument_prompt)

=== Prompts ===
Task: fallacy, Template: fallacy task. Topic: {} Text: {} This contains the fallacy: <mask>
Task: stance, Template: procon task. Topic: {} Text: {} Has the relation: <mask>


### Load Model
We load a model we have already trained for this task. Some sample predictions are also displayed

In [97]:
pmodel = RobertaPrompt(model='/content/drive/MyDrive/Laidlaw Research Project/models/prompt_combined', device = torch.device('cuda'), prompt = argument_prompt)

In [98]:
print(pmodel)

======== Base Model ============
/content/drive/MyDrive/Laidlaw Research Project/models/prompt_combined

======== Tasks ============
Task: fallacy, Template: fallacy task. Topic: {} Text: {} This contains the fallacy: <mask>
Task: stance, Template: procon task. Topic: {} Text: {} Has the relation: <mask>



In [99]:
fallacy = pmodel.infer(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: NoFallacy
Stance: contradict


In [100]:
fallacy = pmodel.infer(["Should we allow animal testing?", "Your stupid for bringing this up, animal testing is horrible"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "Your stupid for bringing this up, animal testing is horrible"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: AdHominem
Stance: contradict


In [101]:
fallacy = pmodel.infer(["Should we allow animal testing?", "My Dad had a dog once, he says animal testing should be allowed"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "My Dad had a dog once, he says animal testing should be allowed"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: IrrelevantAuthority
Stance: support


In [102]:
fallacy = pmodel.infer(["Should we allow animal testing?", "Everyone has a favorite animal, what is yours?"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "Everyone has a favorite animal, what is yours?"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: RedHerring
Stance: support


In [103]:
print(pmodel.test("/content/drive/MyDrive/Laidlaw Research Project/data/test_samples.tsv"))

macro f1: 0.7863247863247864
micro f1: 0.7933884297520661
weighted f1: 0.7943516538557861
              precision    recall  f1-score   support

  contradict       0.73      0.77      0.75        48
     support       0.84      0.81      0.83        73

    accuracy                           0.79       121
   macro avg       0.78      0.79      0.79       121
weighted avg       0.80      0.79      0.79       121



## Train Model

In [ ]:
pmodel = RobertaPrompt(model='roberta-large', device = torch.device('cuda'), prompt = argument_prompt)

In [ ]:
pmodel.train("sample_train_set.tsv", "sample_val_set.tsv", output_dir="sample_model", epochs=10)

In [ ]:
pmodel.test("sample_test_set.tsv")